# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [1]:
from datetime import datetime

# Note that tstart and tend should span for at least one day because of the way the scripts are written.
tstart = '2018-07-18T00:00:00'
tend = '2018-07-19T00:00:00'

ttarget = '2018-07-19T00:51:00'
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')
ttarget = datetime.strptime(ttarget,'%Y-%m-%dT%H:%M:%S')



---
# Step 1, get the nominal RA/Dec Position for the Moon at the nominal target time

## We use this to determine what the occultation times are.

In [2]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(ttarget)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3:03d}:00:00:00 {4}:{5:03d}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



[#################################] 100% deltat.preds


../orbit_engine/lunar_199to200.sh


---
# Step 2, get the PA position 

## We use this to figure out how to offset the FoV for the Moon

In [3]:

pastring = "./att2pa {0} {1} {2} -5 > lunar_{3}_{4}.pa".format(
    ttarget.strftime('%Y:%j:%H:%M:%S'),
    ra.to(u.deg).value, dec.to(u.deg).value,
    tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
outfile = '../orbit_engine/lunar_{0}to{1}_pa.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(pastring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)


../orbit_engine/lunar_199to200_pa.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced to produce the occultation files:

`./lunar_225to226.sh`

...and the one to get the PA position. **Remember that this is the MPS PA angle, so the SKY PA = 180 - MPA_PA. However, the code in nsutar_planning.io knows how to fix this and will return SKY PA values.**

`./lunar_225to226_pa.sh`

Check in the resulting output files in to github if you're working remotely.

`git add lunar_225to226.occ lunar_225to226.pa`

`git commit -m "Files for days 225-226"`

`git push`

Then pull this down to your local environment and continue below.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [4]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )
pafile= "../orbit_engine/lunar_{0}_{1}.pa".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

orbits = io.parse_occ(occfile)
sky_pa = io.parse_pa(pafile)


# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()
print('SKY PA angle: ', sky_pa)

SKY PA angle:  -3.404853000000003


# Use SkyField to get the location of the Moon for each orbit:

This puts the output into the provided text file. This shows the *aim* time that was used to determine the pointing. You should slew while the source is occulted.

In [5]:
# Sanity check. Use this block to figure out which orbit(s) in the occultation file you want to use

import nustar_planning.lunar_planning as lunar_planning
sub = orbits.iloc[0:1].reset_index()
lunar_planning.position(sub, steps = 5, parallax_correction=True, load_path ='../../data')

[#################################] 100% deltat.preds
[#################################] 100% deltat.preds


Aim Time            RA         Dec
2018-07-18T00:27:42 185.12058  2.99540
2018-07-18T00:38:51.400000 184.97936  3.06856
2018-07-18T00:50:00.800000 184.41206  3.12604
2018-07-18T01:01:10.200000 183.76254  3.11751
2018-07-18T01:12:19.600000 183.42279  3.02904


In [13]:
# PA == sky_pa from above, Rmoon = 940 arcseconds (default), 6 arcminutes maximum drift (default) and
# no minimum dwell period (default)

# Note, this takes a few minutes to run!

outfile = 'lunar_{0}_{1}_pointing_test.txt'.format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

lunar_planning.position_shift(sub, parallax_correction=True, pa = sky_pa*u.deg, dt = 10*u.s, min_shift=8*u.arcmin,
                              outfile=outfile,pad_time=0*u.s, load_path ='../../data')
print("Output is stored in: {}".format(outfile))

[#################################] 100% deltat.preds


Output is stored in: lunar_199_200_pointing_test.txt


In [7]:
import numpy as np
from astropy.coordinates import SkyCoord

outfile = 'lunar_{0}_{1}_pointing_test.txt'.format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )


f = open(outfile, 'r')
times = []
sky_pos = []
for line in f:
    if line.startswith('A'):
        continue



    fields = line.split()
    this_time = datetime.strptime(fields[0], '%Y:%j:%H:%M:%S')

    times = np.append(times, this_time)
    
    this_skypos = SkyCoord(ra=fields[1], dec=fields[2], unit ='deg')
    sky_pos = np.append(sky_pos, this_skypos)
    
f.close()

good_exp = 0.
all_exp = 0.
for ind in range(len(times)):
    if ind == 0:
        continue
    dt = (times[ind] - times[ind-1]).total_seconds()
    all_exp += dt
#    if (dt < 200):
#        continue
    dr = sky_pos[ind].separation(sky_pos[ind-1]).arcmin
    print(dt, dr)
    good_exp += dt

print()
print(good_exp, all_exp)


630.0 8.333919390135684
210.0 8.350363753285741
170.0 8.277327912806248
150.0 8.257179871578842
140.0 8.539562512061337
140.0 8.589069771103606
140.0 8.405272300945134
140.0 8.541972525298863
150.0 8.295331636973433
160.0 8.376738027103292
190.0 8.009292914794482
250.0 8.125187494869097

2470.0 2470.0
